In [4]:
import numpy as np
from collections import defaultdict
import struct
import zlib
import base64
import os
import re
from datetime import datetime
from google.colab import files
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output

# ============================================================
# CORE COMPRESSION ENGINE
# ============================================================

class MetaMarkovCompressor:
    """Base adaptive Markov compressor"""

    def __init__(self, order=3):
        self.order = order

    def compress_to_ranks(self, text):
        """Compress text to list of ranks + metadata"""
        if len(text) <= self.order:
            return {
                'prefix': text,
                'ranks': [],
                'max_values': [],
                'alphabet': list(set(text)) if text else []
            }

        transitions = defaultdict(lambda: defaultdict(int))
        alphabet = sorted(set(text))
        char_to_idx = {c: i for i, c in enumerate(alphabet)}

        prefix = text[:self.order]
        ranks = []
        max_values = []

        for i in range(self.order, len(text)):
            context = text[i-self.order:i]
            current_char = text[i]

            if context in transitions:
                seen_chars = sorted(
                    transitions[context].items(),
                    key=lambda x: (-x[1], x[0])
                )
                seen_list = [c for c, _ in seen_chars]

                if current_char in seen_list:
                    rank = seen_list.index(current_char)
                    max_val = len(seen_list) + 1
                else:
                    rank = len(seen_list)
                    max_val = len(seen_list) + 1

                ranks.append(rank)
                max_values.append(max_val)

                if current_char not in seen_list:
                    unseen = [c for c in alphabet if c not in seen_list]
                    unseen_idx = unseen.index(current_char)
                    ranks.append(unseen_idx)
                    max_values.append(len(unseen))
            else:
                rank = char_to_idx[current_char]
                ranks.append(rank)
                max_values.append(len(alphabet))

            transitions[context][current_char] += 1

        return {
            'prefix': prefix,
            'ranks': ranks,
            'max_values': max_values,
            'alphabet': alphabet,
            'original_len': len(text)
        }

    def decompress_from_ranks(self, data):
        """Reconstruct text from ranks"""
        prefix = data['prefix']
        ranks = data['ranks']
        max_values = data['max_values']
        alphabet = data['alphabet']
        original_len = data['original_len']

        if not ranks:
            return prefix

        result = list(prefix)
        transitions = defaultdict(lambda: defaultdict(int))

        rank_idx = 0

        while len(result) < original_len and rank_idx < len(ranks):
            context = ''.join(result[-self.order:])

            if context in transitions:
                seen_chars = sorted(
                    transitions[context].items(),
                    key=lambda x: (-x[1], x[0])
                )
                seen_list = [c for c, _ in seen_chars]

                rank = ranks[rank_idx]
                rank_idx += 1

                if rank < len(seen_list):
                    char = seen_list[rank]
                else:
                    unseen = [c for c in alphabet if c not in seen_list]
                    unseen_idx = ranks[rank_idx]
                    rank_idx += 1
                    char = unseen[unseen_idx]
            else:
                rank = ranks[rank_idx]
                rank_idx += 1
                char = alphabet[rank]

            result.append(char)
            transitions[context][char] += 1

        return ''.join(result)


class OmegaInfinityCompressor:
    """
    Omega Infinity v4.1 - Meta-Recursive Compression
    Fixed version with proper base64 handling
    """

    VERSION = "4.1"
    MAGIC_BYTES = b'OMEGA'

    def __init__(self, base_order=4, meta_levels=3, verbose=True):
        self.base_order = base_order
        self.meta_levels = meta_levels
        self.verbose = verbose

    def log(self, message, **kwargs):
        if self.verbose:
            print(message, **kwargs)

    def compress(self, data, is_binary=False):
        """Compress data (text or binary)"""
        self.log(f"{'='*60}")
        self.log(f"🌌 OMEGA INFINITY v{self.VERSION} - COMPRESSION")
        self.log(f"{'='*60}")

        # Convert to text representation if binary
        if is_binary or isinstance(data, bytes):
            text = base64.b64encode(data if isinstance(data, bytes) else data.encode()).decode('ascii')
            data_type = 'binary'
        else:
            text = data
            data_type = 'text'

        original_size = len(text.encode('utf-8'))
        self.log(f"📝 Original size: {original_size:,} bytes")
        self.log(f"📊 Base order: {self.base_order}")
        self.log(f"🔄 Meta-levels: {self.meta_levels}")
        self.log(f"📁 Data type: {data_type}")
        self.log(f"{'='*60}\n")

        levels_data = []
        current_text = text

        for level in range(self.meta_levels):
            self.log(f"⚙️  Level {level + 1}:")
            self.log(f"   📥 Input: {len(current_text):,} symbols")

            order = max(1, self.base_order - level)
            compressor = MetaMarkovCompressor(order=order)

            data_compressed = compressor.compress_to_ranks(current_text)

            self.log(f"   📦 Ranks generated: {len(data_compressed['ranks']):,}")
            self.log(f"   📉 Max rank: {max(data_compressed['ranks']) if data_compressed['ranks'] else 0}")
            self.log(f"   🔤 Alphabet: {len(data_compressed['alphabet'])} symbols")

            level_meta = {
                'order': order,
                'prefix': data_compressed['prefix'],
                'alphabet': data_compressed['alphabet'],
                'max_values': data_compressed['max_values'],
                'original_len': data_compressed['original_len']
            }
            levels_data.append(level_meta)

            if data_compressed['ranks']:
                max_rank = max(data_compressed['ranks'])
                if max_rank < 256:
                    current_text = ''.join(chr(r) for r in data_compressed['ranks'])
                else:
                    current_text = ','.join(map(str, data_compressed['ranks']))
            else:
                current_text = ""
                break

            self.log(f"   📤 Output: {len(current_text):,} symbols")

            if len(current_text) < 10:
                self.log(f"   ⏹️  Small enough, stopping.")
                break

            self.log("")

        final_ranks = [ord(c) for c in current_text] if current_text else []

        package = self._pack_all(levels_data, final_ranks, data_type, original_size)
        compressed = zlib.compress(package, level=9)

        header = self.MAGIC_BYTES + struct.pack('B', int(float(self.VERSION.split('.')[0])))
        final_compressed = header + compressed

        compressed_size = len(final_compressed)
        ratio = (1 - compressed_size / original_size) * 100

        self.log(f"\n{'='*60}")
        self.log(f"📊 FINAL RESULTS")
        self.log(f"{'='*60}")
        self.log(f"📁 Original: {original_size:,} bytes")
        self.log(f"📦 Compressed: {compressed_size:,} bytes")
        self.log(f"📉 Ratio: {ratio:.2f}%")
        self.log(f"🔢 Factor: {original_size/compressed_size:.2f}x")
        self.log(f"🔄 Levels used: {len(levels_data)}")
        self.log(f"{'='*60}")

        return final_compressed

    def decompress(self, compressed_data):
        """Decompress data"""
        self.log(f"\n{'='*60}")
        self.log(f"🔓 OMEGA INFINITY - DECOMPRESSION")
        self.log(f"{'='*60}")

        if not compressed_data.startswith(self.MAGIC_BYTES):
            raise ValueError("Invalid Omega Infinity file format! Missing OMEGA header.")

        version = struct.unpack('B', compressed_data[5:6])[0]
        self.log(f"📋 File version: {version}")

        data = zlib.decompress(compressed_data[6:])
        pos = 0

        data_type_len = struct.unpack('B', data[pos:pos+1])[0]; pos += 1
        data_type = data[pos:pos+data_type_len].decode('utf-8'); pos += data_type_len

        original_size = struct.unpack('I', data[pos:pos+4])[0]; pos += 4

        num_levels = data[pos]; pos += 1
        self.log(f"🔄 Levels to process: {num_levels}")
        self.log(f"📁 Data type: {data_type}")

        levels_data = []
        for i in range(num_levels):
            level = {}

            level['order'] = data[pos]; pos += 1
            level['original_len'] = struct.unpack('I', data[pos:pos+4])[0]; pos += 4

            prefix_len = struct.unpack('H', data[pos:pos+2])[0]; pos += 2
            level['prefix'] = data[pos:pos+prefix_len].decode('utf-8'); pos += prefix_len

            alphabet_len = struct.unpack('H', data[pos:pos+2])[0]; pos += 2
            level['alphabet'] = list(data[pos:pos+alphabet_len].decode('utf-8')); pos += alphabet_len

            num_max_values = struct.unpack('I', data[pos:pos+4])[0]; pos += 4
            level['max_values'] = []
            for _ in range(num_max_values):
                mv = struct.unpack('H', data[pos:pos+2])[0]; pos += 2
                level['max_values'].append(mv)

            levels_data.append(level)

        num_final = struct.unpack('I', data[pos:pos+4])[0]; pos += 4
        final_ranks = []
        for _ in range(num_final):
            r = struct.unpack('H', data[pos:pos+2])[0]; pos += 2
            final_ranks.append(r)

        current_ranks = final_ranks

        for i in range(num_levels - 1, -1, -1):
            level = levels_data[i]
            # Changed from **kwargs to end="" to fix NameError
            self.log(f"   ⬆️  Level {i + 1}: {len(current_ranks):,} ranks → ", end="")

            decomp_data = {
                'prefix': level['prefix'],
                'ranks': current_ranks,
                'max_values': level['max_values'],
                'alphabet': level['alphabet'],
                'original_len': level['original_len']
            }

            compressor = MetaMarkovCompressor(order=level['order'])
            text = compressor.decompress_from_ranks(decomp_data)

            self.log(f"{len(text):,} symbols")

            if i > 0:
                current_ranks = [ord(c) for c in text]
            else:
                result = text

        if data_type == 'binary':
            result = base64.b64decode(result)

        final_size = len(result) if isinstance(result, bytes) else len(result.encode('utf-8'))
        self.log(f"\n✅ Recovered: {final_size:,} bytes")
        self.log(f"{'='*60}")

        return result

    def _pack_all(self, levels_data, final_ranks, data_type, original_size):
        """Pack all levels into bytes"""
        package = bytearray()

        data_type_bytes = data_type.encode('utf-8')
        package.append(len(data_type_bytes))
        package.extend(data_type_bytes)

        package.extend(struct.pack('I', original_size))
        package.append(len(levels_data))

        for level in levels_data:
            package.append(level['order'])
            package.extend(struct.pack('I', level['original_len']))

            prefix_bytes = level['prefix'].encode('utf-8')
            package.extend(struct.pack('H', len(prefix_bytes)))
            package.extend(prefix_bytes)

            alphabet_str = ''.join(level['alphabet'])
            alphabet_bytes = alphabet_str.encode('utf-8')
            package.extend(struct.pack('H', len(alphabet_bytes)))
            package.extend(alphabet_bytes)

            package.extend(struct.pack('I', len(level['max_values'])))
            for mv in level['max_values']:
                package.extend(struct.pack('H', min(mv, 65535)))

        package.extend(struct.pack('I', len(final_ranks)))
        for r in final_ranks:
            package.extend(struct.pack('H', r))

        return bytes(package)

    # ===== ENCODING/DECODING HELPERS =====

    @staticmethod
    def to_base64(data):
        """Convert compressed bytes to clean base64 string"""
        b64 = base64.b64encode(data).decode('ascii')
        return b64

    @staticmethod
    def from_base64(b64_string):
        """Convert base64 string back to bytes (with cleaning)"""
        # Remove all whitespace, newlines, etc.
        cleaned = re.sub(r'\s+', '', b64_string.strip())

        # Fix padding if needed
        missing_padding = len(cleaned) % 4
        if missing_padding:
            cleaned += '=' * (4 - missing_padding)

        return base64.b64decode(cleaned)

    def compress_to_string(self, text):
        """Compress text and return as base64 string"""
        compressed = self.compress(text)
        return self.to_base64(compressed)

    def decompress_from_string(self, b64_string):
        """Decompress from base64 string"""
        compressed = self.from_base64(b64_string)
        return self.decompress(compressed)


# ============================================================
# SIMPLE INTERFACE FUNCTIONS
# ============================================================

def omega_compress(text, order=4, levels=3, verbose=True):
    """
    Compress text and return base64 string

    Args:
        text: Text to compress
        order: Markov order (1-8)
        levels: Meta-levels (1-5)
        verbose: Print progress

    Returns:
        Base64 encoded compressed string
    """
    comp = OmegaInfinityCompressor(base_order=order, meta_levels=levels, verbose=verbose)
    return comp.compress_to_string(text)


def omega_decompress(compressed_string, verbose=True):
    """
    Decompress from base64 string

    Args:
        compressed_string: Base64 encoded compressed data
        verbose: Print progress

    Returns:
        Original text
    """
    comp = OmegaInfinityCompressor(verbose=verbose)
    return comp.decompress_from_string(compressed_string)


def omega_compress_file(filepath, order=4, levels=3):
    """Compress a file and download the result"""
    comp = OmegaInfinityCompressor(base_order=order, meta_levels=levels)

    with open(filepath, 'rb') as f:
        data = f.read()

    compressed = comp.compress(data, is_binary=True)
    output_path = filepath + '.omega'

    with open(output_path, 'wb') as f:
        f.write(compressed)

    files.download(output_path)
    return output_path


def omega_decompress_file(filepath):
    """Decompress a .omega file and download the result"""
    comp = OmegaInfinityCompressor()

    with open(filepath, 'rb') as f:
        compressed = f.read()

    decompressed = comp.decompress(compressed)

    if filepath.endswith('.omega'):
        output_path = filepath[:-6]
    else:
        output_path = filepath + '.decompressed'

    mode = 'wb' if isinstance(decompressed, bytes) else 'w'
    with open(output_path, mode) as f:
        f.write(decompressed)

    files.download(output_path)
    return output_path


# ============================================================
# INTERACTIVE UI
# ============================================================

class OmegaUI:
    """Simple interactive UI"""

    def __init__(self):
        self.comp = OmegaInfinityCompressor(verbose=True)
        self.last_result = None
        self.create_widgets()

    def create_widgets(self):
        # Header
        self.header = HTML("""
        <div style="background: linear-gradient(135deg, #0f0f23 0%, #1a1a3e 100%);
                    padding: 20px; border-radius: 15px; text-align: center; margin-bottom: 20px;">
            <h1 style="color: #e94560; margin: 0; font-size: 2em;">🌌 OMEGA INFINITY v4.1</h1>
            <p style="color: #888; margin: 5px 0;">Meta-Recursive Markov Chain Compression</p>
        </div>
        """)

        # Mode tabs
        self.mode = widgets.ToggleButtons(
            options=['🗜️ Compress', '📂 Decompress'],
            value='🗜️ Compress',
            description='',
            button_style='info'
        )
        self.mode.observe(self.on_mode_change, names='value')

        # Parameters
        self.order = widgets.IntSlider(value=4, min=1, max=8, description='Order:',
                                        style={'description_width': '60px'})
        self.levels = widgets.IntSlider(value=3, min=1, max=5, description='Levels:',
                                         style={'description_width': '60px'})
        self.params = widgets.HBox([self.order, self.levels])

        # Input
        self.input_label = widgets.HTML("<b>📝 Enter text to compress:</b>")
        self.text_input = widgets.Textarea(
            placeholder='Type or paste your text here...',
            layout=widgets.Layout(width='100%', height='200px')
        )

        # Buttons
        self.run_btn = widgets.Button(description='🚀 COMPRESS', button_style='success',
                                       layout=widgets.Layout(width='150px', height='45px'))
        self.run_btn.on_click(self.on_run)

        self.copy_btn = widgets.Button(description='📋 Copy Result', button_style='info',
                                        layout=widgets.Layout(width='150px', height='45px'))
        self.copy_btn.on_click(self.on_copy)

        self.download_btn = widgets.Button(description='💾 Download', button_style='warning',
                                            layout=widgets.Layout(width='150px', height='45px'))
        self.download_btn.on_click(self.on_download)

        self.buttons = widgets.HBox([self.run_btn, self.copy_btn, self.download_btn])

        # Output
        self.output = widgets.Output()
        self.result_label = widgets.HTML("<b>📤 Result:</b>")
        self.result_box = widgets.Textarea(
            placeholder='Result will appear here...',
            layout=widgets.Layout(width='100%', height='200px')
        )

        # Stats
        self.stats = widgets.HTML("")

    def on_mode_change(self, change):
        if change['new'] == '🗜️ Compress':
            self.input_label.value = "<b>📝 Enter text to compress:</b>"
            self.text_input.placeholder = 'Type or paste your text here...'
            self.run_btn.description = '🚀 COMPRESS'
            self.run_btn.button_style = 'success'
        else:
            self.input_label.value = "<b>📦 Paste compressed data (base64):</b>"
            self.text_input.placeholder = 'Paste the base64 compressed string here...'
            self.run_btn.description = '🔓 DECOMPRESS'
            self.run_btn.button_style = 'warning'

        self.result_box.value = ''
        self.stats.value = ''

    def on_run(self, btn):
        with self.output:
            clear_output()

            text = self.text_input.value
            if not text.strip():
                print("⚠️ Please enter some text!")
                return

            try:
                self.comp = OmegaInfinityCompressor(
                    base_order=self.order.value,
                    meta_levels=self.levels.value,
                    verbose=True
                )

                if self.mode.value == '🗜️ Compress':
                    # Compress
                    result = self.comp.compress_to_string(text)
                    self.result_box.value = result
                    self.last_result = result

                    original = len(text.encode('utf-8'))
                    compressed = len(self.comp.from_base64(result))
                    ratio = (1 - compressed/original) * 100

                    self.stats.value = f"""
                    <div style="background: #1a1a2e; padding: 15px; border-radius: 10px; margin-top: 10px;">
                        <h4 style="color: #4ecca3; margin: 0 0 10px 0;">✅ Compression Complete!</h4>
                        <table style="color: white;">
                            <tr><td>📁 Original:</td><td><b>{original:,} bytes</b></td></tr>
                            <tr><td>📦 Compressed:</td><td><b>{compressed:,} bytes</b></td></tr>
                            <tr><td>📉 Ratio:</td><td><b>{ratio:.1f}%</b></td></tr>
                            <tr><td>🔢 Factor:</td><td><b>{original/compressed:.2f}x</b></td></tr>
                        </table>
                        <p style="color: #888; font-size: 12px; margin-top: 10px;">
                            💡 Copy the result above to decompress later!
                        </p>
                    </div>
                    """
                else:
                    # Decompress
                    result = self.comp.decompress_from_string(text)
                    self.result_box.value = result
                    self.last_result = result

                    self.stats.value = f"""
                    <div style="background: #1a1a2e; padding: 15px; border-radius: 10px; margin-top: 10px;">
                        <h4 style="color: #4ecca3; margin: 0 0 10px 0;">✅ Decompression Complete!</h4>
                        <p style="color: white;">📄 Recovered: <b>{len(result):,} characters</b></p>
                    </div>
                    """

            except Exception as e:
                print(f"❌ Error: {str(e)}")
                import traceback
                traceback.print_exc()

    def on_copy(self, btn):
        # In Colab, we can't directly copy to clipboard, so we just select the text
        print("📋 Select and copy the result from the output box above!")

    def on_download(self, btn):
        if not self.last_result:
            print("⚠️ No result to download!")
            return

        if self.mode.value == '🗜️ Compress':
            filename = 'compressed.omega.txt'
            with open(filename, 'w') as f:
                f.write(self.last_result)
        else:
            filename = 'decompressed.txt'
            with open(filename, 'w') as f:
                f.write(self.last_result)

        files.download(filename)

    def show(self):
        display(self.header)
        display(self.mode)
        display(self.params)
        display(self.input_label)
        display(self.text_input)
        display(self.buttons)
        display(self.output)
        display(self.result_label)
        display(self.result_box)
        display(self.stats)


# ============================================================
# QUICK TEST
# ============================================================

def quick_test():
    """Quick test to verify compression/decompression works"""
    print("🧪 Running quick test...")
    print("="*50)

    test_text = "Hello World! This is a test of Omega Infinity compression. " * 10

    print(f"📝 Original: {len(test_text)} chars")

    # Compress
    comp = OmegaInfinityCompressor(verbose=False)
    compressed_b64 = comp.compress_to_string(test_text)

    print(f"📦 Compressed (base64): {len(compressed_b64)} chars")
    print(f"📦 First 50 chars: {compressed_b64[:50]}...")

    # Decompress
    recovered = comp.decompress_from_string(compressed_b64)

    print(f"📄 Recovered: {len(recovered)} chars")

    # Verify
    if test_text == recovered:
        print("✅ TEST PASSED! Compression/Decompression works correctly!")
    else:
        print("❌ TEST FAILED! Data mismatch!")

    print("="*50)
    return test_text == recovered


# ============================================================
# MAIN
# ============================================================

print("""
╔═══════════════════════════════════════════════════════════════╗
║                                                               ║
║      🌌 OMEGA INFINITY v4.1 - Fixed Base64 Version 🌌         ║
║                                                               ║
╚═══════════════════════════════════════════════════════════════╝
""")

# Run quick test first
quick_test()

print("\n")

# Show UI
ui = OmegaUI()
ui.show()

# Also show simple usage examples
print("\n" + "="*60)
print("💡 SIMPLE COMMAND USAGE:")
print("="*60)
print("""
# Compress text:
compressed = omega_compress("Your text here...")

# Decompress:
original = omega_decompress(compressed)

# Compress file:
uploaded = files.upload()
omega_compress_file('yourfile.txt')

# Decompress file:
uploaded = files.upload()  # Upload .omega file
omega_decompress_file('yourfile.omega')
""")


╔═══════════════════════════════════════════════════════════════╗
║                                                               ║
║      🌌 OMEGA INFINITY v4.1 - Fixed Base64 Version 🌌         ║
║                                                               ║
╚═══════════════════════════════════════════════════════════════╝

🧪 Running quick test...
📝 Original: 590 chars
📦 Compressed (base64): 332 chars
📦 First 50 chars: T01FR0EEeNrtl81ugkAUhc8MKIo6gMjfrn2BPgd2oRsT10bRNk...
📄 Recovered: 590 chars
✅ TEST PASSED! Compression/Decompression works correctly!




ToggleButtons(button_style='info', options=('🗜️ Compress', '📂 Decompress'), value='🗜️ Compress')

HTML(value='<b>📝 Enter text to compress:</b>')

Textarea(value='', layout=Layout(height='200px', width='100%'), placeholder='Type or paste your text here...')

Output()

HTML(value='<b>📤 Result:</b>')

Textarea(value='', layout=Layout(height='200px', width='100%'), placeholder='Result will appear here...')

HTML(value='')


💡 SIMPLE COMMAND USAGE:

# Compress text:
compressed = omega_compress("Your text here...")

# Decompress:
original = omega_decompress(compressed)

# Compress file:
uploaded = files.upload()
omega_compress_file('yourfile.txt')

# Decompress file:
uploaded = files.upload()  # Upload .omega file
omega_decompress_file('yourfile.omega')

